In [1]:
import pandas as pd
from google.oauth2 import service_account
from google.cloud import bigquery
import pyodbc
import logging
import numpy as np
from sqlalchemy import create_engine, event
import urllib
from sqlalchemy.types import Integer, String, Unicode, Boolean, DateTime, Float, Numeric

In [2]:
def get_data(dataproject, datasetprojectbi, table, clausule = "", columns = '*'):
    query = f"SELECT {columns} FROM `{dataproject}.{datasetprojectbi}.{table}` {clausule}"
    datatable = pd.read_gbq(
            query,
            project_id=dataproject,
            dialect="standard")
    return datatable

In [3]:
server = '10.34.1.4' 
database = 'AMB_BIGQUERY' 
username = 'BIGQUERY_BI' 
password = 'AN2 6optimist hurts a backpack'
db_params = urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))

In [7]:
@event.listens_for(engine, 'before_cursor_execute')
def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
    if executemany:
        cursor.fast_executemany = True

In [8]:
dfdate = get_data(dataproject = "bigqueryunilever", datasetprojectbi = "infrashop_clean", table = "Tabela_Calculo_6_Fato", columns = 'DISTINCT CAST(DT_INS AS DATE)')

dfdate['NaT'] = dfdate['f0_'].apply(lambda x: True if type(x) == pd._libs.tslibs.nattype.NaTType else False)

dfdate = dfdate[dfdate['NaT']!= True].drop('NaT', axis =1).sort_values(by=['f0_'])

for date in dfdate['f0_']:
    date = str(date).replace(' 00:00:00','')
    print(f"lendo FAT_PEDIDO_FATURADO BigQUery: {date}")
    df1 = get_data("bigqueryunilever", "infrashop_clean", "Tabela_Calculo_6_Fato", clausule = f"Where CAST(DT_INS AS DATE) = '{date}'")
    columns=['SK_PEDIDO', 'SK_FRONT', 'SK_PRODUTO', 'SK_KIT', 'SK_DATA', 'SK_CANAL_VENDA',
             'SK_ENDERECO', 'SK_CLIENTE', 'SK_DISTRIBUIDOR', 'SK_TIPO_PAGAMENTO', 'CD_NOTA_FISCAL',
             'QTD_ITEM_NF', 'VLR_ITEM_NF', 'VLR_TOTAL_NF', 'QTD_ITEM_A_ITEM_PS', 'QTD_ITEM_A_ITEM_PSK',
             'VLR_ITEM_A_ITEM_PS', 'VLR_ITEM_A_ITEM_PSK', 'FATOR_QTD_PS', 'FATOR_QTD_PSK',
             'FATOR_BONIFICADO_PS', 'FATOR_BONIFICADO_PSK', 'FLG_ORIGEM']
    df1[columns] = df1[columns].fillna(-2)
    df1["DT_INS"] = df1["DT_INS"].astype('datetime64[ns]')
    print(f"exluindo FAT_PEDIDO_FATURADO: {date} do servidor")
    engine.execute(f"DELETE FROM FAT_PEDIDO_FATURADO_TESTE WHERE FLG_ORIGEM=3 and CAST(DT_INS AS DATE) = '{date}'")
    print(f"escrevendo FAT_PEDIDO_FATURADO: {date} no servidor")
    engine.fast_executemany = True
    df1.to_sql('FAT_PEDIDO_FATURADO_TESTE',con=engine,index=False,if_exists="append",schema="dbo",
           dtype={   "CD_NOTA_FISCAL": Integer(),
                     "QTD_ITEM_NF": Integer(),
                     "VLR_ITEM_NF": Numeric(10,2),
                     "VLR_TOTAL_NF": Numeric(10,2),
                     "QTD_ITEM_A_ITEM_PS": Numeric(10,2),
                     "QTD_ITEM_A_ITEM_PSK": Numeric(10,2),
                     "VLR_ITEM_A_ITEM_PS": Numeric(10,2),
                     "VLR_ITEM_A_ITEM_PSK": Numeric(10,2),
                     "FATOR_QTD_PS": Numeric(10,4),
                     "FATOR_QTD_PSK": Numeric(10,4),
                     "FATOR_BONIFICADO_PS": Numeric(10,4),
                     "FATOR_BONIFICADO_PSK": Numeric(10,4),
                     "DT_INS": DateTime(),
                     "FLG_ORIGEM": Integer()})


Downloading: 100%|██████████| 33/33 [00:01<00:00, 25.46rows/s]


lendo FAT_PEDIDO_FATURADO BigQUery: 2020-01-27


Downloading: 100%|██████████| 4558/4558 [00:02<00:00, 1722.12rows/s]


exluindo FAT_PEDIDO_FATURADO: 2020-01-27 do servidor
escrevendo FAT_PEDIDO_FATURADO: 2020-01-27 no servidor
lendo FAT_PEDIDO_FATURADO BigQUery: 2020-01-28


Downloading: 100%|██████████| 3376/3376 [00:02<00:00, 1475.53rows/s]


...

exluindo FAT_PEDIDO_FATURADO: 2020-03-12 do servidor
escrevendo FAT_PEDIDO_FATURADO: 2020-03-12 no servidor
lendo FAT_PEDIDO_FATURADO BigQUery: 2020-03-13


Downloading: 100%|██████████| 2331/2331 [00:01<00:00, 1276.19rows/s]


exluindo FAT_PEDIDO_FATURADO: 2020-03-13 do servidor
escrevendo FAT_PEDIDO_FATURADO: 2020-03-13 no servidor
lendo FAT_PEDIDO_FATURADO BigQUery: 2020-03-14


Downloading: 100%|██████████| 13243/13243 [00:04<00:00, 2837.21rows/s]


exluindo FAT_PEDIDO_FATURADO: 2020-03-14 do servidor
escrevendo FAT_PEDIDO_FATURADO: 2020-03-14 no servidor
